# Data Science Programming

**Oleh:** \
2105551068 - Ni Made Deni Sikiandani \
2105551126 - I Nyoman Yodya Mahesa Sastra

**Dokumentasi Google Speech-to-Text API:** \
Basics: https://cloud.google.com/speech-to-text/v2/docs/basics \
Model: https://cloud.google.com/speech-to-text/v2/docs/transcription-model \
Synchronous: https://cloud.google.com/speech-to-text/v2/docs/sync-recognize \
Asynchronous: https://cloud.google.com/speech-to-text/v2/docs/batch-recognize \
Streaming: https://cloud.google.com/speech-to-text/v2/docs/streaming-recognize

**Install Library:**\
```pip install google-cloud-speech```\
```pip install moviepy```


## Synchronous

In [6]:
# Limitasi Synchronous:
# ---------------------------------
# Audio Maximum 1 Menit
# Audio Maximum 10 MB
# ---------------------------------

# Import Library
from google.cloud.speech_v2 import SpeechClient
from google.cloud.speech_v2.types import cloud_speech
from google.oauth2 import service_account


# Function transkrip audio Synchronous
def transcribe_file_v2(project_id: str, audio_file: str) -> cloud_speech.RecognizeResponse:
    # Instantiates client
    credentials = service_account.Credentials.from_service_account_file('gcloud_apikey.json')
    client = SpeechClient(credentials=credentials)

    # Baca file dalam bytes
    with open(audio_file, "rb") as f:
        content = f.read()

    # Config untuk speech recognition
    config = cloud_speech.RecognitionConfig(
        auto_decoding_config=cloud_speech.AutoDetectDecodingConfig(),
        language_codes=["en-US"],
        model="long",
    )

    # Membentuk objek request
    request = cloud_speech.RecognizeRequest(
        recognizer=f"projects/{project_id}/locations/global/recognizers/_",
        config=config,
        content=content,
    )

    # Transkrip audio menjadi teks
    response = client.recognize(request=request)

    # Print respon transkrip
    for result in response.results:
        print(f"Transcript: {result.alternatives[0].transcript}")

    # Return respon transkrip
    return response

## Asynchronous


In [7]:
# Limitasi Synchronous:
# ---------------------------------
# Audio Maximum 480 Menit (8 jam)
# Audio Wajib dari Cloud Storage
# ---------------------------------

## Streaming

In [8]:
# Limitasi Streaming:
# ---------------------------------
# Stream Audio Maximum 5 Menit
# Stream Audio dari Microphone
# ---------------------------------

## Mengambil Transkrip dari Video
Berikut merupakan sepotong scene video **Anime** berjudul **A Whisker Away** di saat Muge memberikan **surat cinta** kepada Hinode, namun rencananya **tidak berjalan sesuai dengan rencana**.

In [9]:
%%HTML
<video width="500" controls>
    <source src="./Anime.mp4" type="video/mp4">
    <track src="Anime.srt" kind="subtitles">
</video>

### 01. Convert Video ke Audio
Tahap pertama yaitu kita **convert** terlebih dahulu **file video** menjadi **file audio**

In [10]:
# Import Library
from moviepy.editor import VideoFileClip

# Deklarasi input dan output
mp4_file = "Anime.mp4"
mp3_file = "Anime.mp3"

# Load video
video_clip = VideoFileClip(mp4_file)

# Ekstrak audio dari video
audio_clip = video_clip.audio

# Simpan audio ke file
audio_clip.write_audiofile(mp3_file)

# Menutup file audio dan video
audio_clip.close()
video_clip.close()

MoviePy - Writing audio in Anime.mp3


MoviePy - Done.


### 02. Transkrip Audio dengan Google Speech API
Tahap kedua yaitu melakukan **transkrip audio** menggunakan **Google Speech API**

In [11]:
## Menyimpan Hasil Transkrip
hasil_transkrip = transcribe_file_v2(
    project_id='data-science-programming-ti24',
    audio_file='Anime.mp3')

Transcript: she know that I I just wanted to cheer you see we you and I are the same I can't say the things that I truly feel just stop it and you're wrong we're nothing alike
Transcript:  it's really embarrassed me
Transcript:  have you considered my feelings at all everyone just hurt this I seriously hate pushy people like you
Transcript:  what if I told you in person just leave me alone didn't you hear me
Transcript:  but I hate you
Transcript:  I see
Transcript:  you hate me that much okay well then
Transcript:  hey


### 03. Menampilkan Detail Hasil Transkrip
Tahap ketiga yaitu menampilkan **detail hasil transkrip**

In [13]:
print(hasil_transkrip)

results {
  alternatives {
    transcript: "she know that I I just wanted to cheer you see we you and I are the same I can\'t say the things that I truly feel just stop it and you\'re wrong we\'re nothing alike"
    confidence: 0.877077401
  }
  result_end_offset {
    seconds: 13
    nanos: 880000000
  }
  language_code: "en-US"
}
results {
  alternatives {
    transcript: " it\'s really embarrassed me"
    confidence: 0.944916487
  }
  result_end_offset {
    seconds: 17
    nanos: 140000000
  }
  language_code: "en-US"
}
results {
  alternatives {
    transcript: " have you considered my feelings at all everyone just hurt this I seriously hate pushy people like you"
    confidence: 0.959103167
  }
  result_end_offset {
    seconds: 25
    nanos: 920000000
  }
  language_code: "en-US"
}
results {
  alternatives {
    transcript: " what if I told you in person just leave me alone didn\'t you hear me"
    confidence: 0.890649438
  }
  result_end_offset {
    seconds: 31
    nanos: 4600

### 04. Menyimpan Hasil Transkrip ke List
Tahap keempat yaitu **menyimpan hasil transkrip** ke bentuk list agar selanjutnya dapat dilakukan tahap **analisis perkataan toxicity** terhadap video tersebut

In [18]:
list_hasil_transkrip = list()

for hasil in hasil_transkrip.results:
    teks = f'{hasil.alternatives[0].transcript}'
    list_hasil_transkrip.append(teks)

list_hasil_transkrip

["she know that I I just wanted to cheer you see we you and I are the same I can't say the things that I truly feel just stop it and you're wrong we're nothing alike",
 " it's really embarrassed me",
 ' have you considered my feelings at all everyone just hurt this I seriously hate pushy people like you',
 " what if I told you in person just leave me alone didn't you hear me",
 ' but I hate you',
 ' I see',
 ' you hate me that much okay well then',
 ' hey']

### 05. Predict Teks

In [5]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization #tokenization|
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from matplotlib import pyplot as plt
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

model = tf.keras.models.load_model('../toxic_comment/toxic.h5')

C:\Users\Yodya Mahesa\scoop\apps\python311\current\Lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


TypeError: Error when deserializing class 'Embedding' using config={'name': 'embedding_2', 'trainable': True, 'dtype': 'float32', 'input_dim': 200001, 'output_dim': 32, 'embeddings_initializer': {'class_name': 'RandomUniform', 'config': {'minval': -0.05, 'maxval': 0.05, 'seed': None}}, 'embeddings_regularizer': None, 'activity_regularizer': None, 'embeddings_constraint': None, 'mask_zero': False, 'input_length': None, 'input_shape': [None, None]}.

Exception encountered: Unrecognized keyword arguments passed to Embedding: {'input_length': None}